In [1]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length

In [2]:
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

In [3]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

init_state = tf.placeholder(tf.float32, [batch_size, state_size])

In [4]:
W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

In [5]:
# Unpack columns
inputs_series = tf.split(batchX_placeholder, truncated_backprop_length, 1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

In [10]:
# Forward passes
cell = tf.contrib.rnn.BasicRNNCell(state_size)
states_series, current_state = cell(inputs_series, init_state)

ValueError: Layer basic_rnn_cell_5 expects 1 inputs, but it received 15 input tensors. Inputs received: [<tf.Tensor 'split:0' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:1' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:2' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:3' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:4' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:5' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:6' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:7' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:8' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:9' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:10' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:11' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:12' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:13' shape=(5, 1) dtype=float32>, <tf.Tensor 'split:14' shape=(5, 1) dtype=float32>]

In [ ]:
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        x,y = generateData()
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })

            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)
                plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()
view raw2-2-rnn-api.py hosted with ❤ by GitHub